In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

In [3]:
def get_urls(base_url):
    URLs = []
    for page in range(1, 51):
        url = base_url.format(page)
        URLs.append(url)
    return URLs

In [4]:
def book_details(book):
    rating_values = {'One':1, 'Two':2, 'Three': 3, 'Four': 4, 'Five':5}
    book_name = book.find('h3').find('a').get('title')
    key = book.find('p', class_='star-rating').get('class')[1]
    ratting = rating_values[key]
    price = float(book.find('p', class_="price_color").text.split('£')[1])
    stock_status = book.find('div', class_="product_price").find_all('p')[1].text.strip()

    return [book_name, ratting, price, stock_status]

In [5]:
def scraping(URL):
    try:
        response = requests.get(URL)
    except:
        print("Provide proper URL")
        return []
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all("li", class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
    Book_Details = []
    for book in books:
        Book_Details.append(book_details(book))
    return Book_Details

In [6]:
all_urls = get_urls(base_url)

In [7]:
all_books = []
for URL in all_urls:
    all_books.extend(scraping(URL))

In [8]:
columns = ['book_name', 'ratting (out of 5)', 'price (in £)', 'stock_status']

In [9]:
df = pd.DataFrame(all_books, columns=columns)

In [10]:
df.head()

,book_name,ratting (out of 5),price (in £),stock_status
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.10,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [11]:
df.shape

(1000, 4)

In [12]:
df.to_csv("Books_Details.csv", index=False)